## Imports

In [1]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gymnasium as gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

## Create the environment

In [2]:
make_one_env = lambda: gym.make("Pendulum-v1")
make_time_limit_env = lambda: gym.wrappers.TimeLimit(make_one_env(), max_episode_steps=config["max_episode_steps"])
make_auto_reset_env = lambda: gym.wrappers.AutoResetWrapper(make_time_limit_env())

env = gym.vector.AsyncVectorEnv([lambda: make_auto_reset_env() for i in range(n_envs)])

env = gym.experimental.wrappers.RescaleActionV0(env, min_action=config["action_space_low"], max_action=config["action_space_high"])
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=config["n_updates"])

In [3]:
# env = make_env()
# 
# env.reset()
# for i in range(1000):
#     obs, reward, terminated, truncated, info = env.step(np.random.rand(5,3)*100)
#     done = [te or tr for te, tr in zip(terminated, truncated)]
#     print(done)

In [4]:
# agent = ContinuousActorCritic()
agent = DiscreteActorCritic(n_features=3, n_actions=1)

# agent.load_weights("weights/ContinuousActorCritic_0")

# vae = VAE()
# vae.optim = optim.Adam(vae.parameters(), lr=1e-4, weight_decay=1e-6)

Initializing agent with 3 features and 1 actions.
Initializing critic.
Adding zero weight init to the output layer.
Initializing actor.


In [5]:
# New training loop with batches for the distributional critic

tracker = MetricsTracker(
    training_metrics=["critic_loss", "actor_loss"],
    episode_metrics=["rewards", "log_probs", "value_preds", "critic_dists", "entropies", "masks"],
)

for sample_phase in tqdm(range(n_updates)):
    
    if sample_phase == 0:
        obs, info = env.reset(seed=42)
        # obs = transform(torch.tensor(obs)) ### only for images

    for step in range(n_steps_per_update):
            
        value_pred, critic_dist = agent.apply_critic(obs)

        # Get an action and take an environment step
        action, log_prob, actor_entropy = agent.get_action(obs)
        obs, reward, terminated, truncated, info = env.step(to_np(action))
        # obs = transform(torch.tensor(obs)) ### only for images
        
        # every step:
        tracker.add(
            episode_metrics={
                "rewards": reward,
                "log_probs": log_prob,
                "value_preds": value_pred,
                "critic_dists": critic_dist,
                "entropies": actor_entropy,
                "masks": np.where(terminated, 0, 1),
            }
        )
    
    # every sample phase:
    episode_batches = tracker.get_episode_batches() # episode_batches is a dict
    last_value_pred, _ = agent.apply_critic(obs) # last value prediction for GAE

    # Update the agent's parameters
    # DEBUG:
    for key, val in episode_batches.items():
        print(key, val.shape)
    critic_loss, actor_loss = agent.get_loss(episode_batches, last_value_pred)
    agent.update_parameters(critic_loss, actor_loss)
    
    # Log the episode metrics
    if sample_phase % config["log_interval"] == 0:
        tracker.add(
            training_metrics={
                "critic_loss": critic_loss,
                "actor_loss": actor_loss,
            }
        )

        # Episode return
        if len(env.return_queue):
            tracker.writer.add_scalar("episode_return", np.array(env.return_queue)[-1], global_step=len(env.return_queue))

        # Actor and critic losses
        tracker.log_to_tensorboard(step=sample_phase)
        

  0%|                                                                                                | 0/500000 [00:00<?, ?it/s]

rewards torch.Size([16, 5])
log_probs torch.Size([16, 5])
value_preds torch.Size([16, 5])
critic_dists torch.Size([16, 5, 255])
entropies torch.Size([16, 5])
masks torch.Size([16, 5])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
ep_rewards = episode_batches["rewards"]
ep_log_probs = episode_batches["log_probs"]
ep_value_preds = episode_batches["value_preds"]
batch_critic_dists = episode_batches["critic_dists"]
ep_entropies = episode_batches["entropies"]
ep_masks = episode_batches["masks"]
last_value_pred = torch.randn(5).to(device)

In [ ]:
print(ep_rewards.shape)
print(last_value_pred.shape)

In [ ]:
last_value_pred = last_value_pred.unsqueeze(0).detach() # (1, B)
ep_value_preds = torch.cat((ep_value_preds, last_value_pred), dim=0) # (SEQ_LEN+1, B)

# set up tensors for the advantage calculation
returns = torch.zeros_like(ep_rewards).to(device) # (SEQ_LEN, B)
advantages = torch.zeros_like(ep_rewards).to(device) # (SEQ_LEN, B)
next_advantage = torch.zeros_like(last_value_pred) # (1, B)

In [ ]:
# calculate advantages using GAE
for t in reversed(range(len(ep_rewards))):
    returns[t] = ep_rewards[t] + gamma * ep_masks[t] * ep_value_preds[t+1]
    td_error = returns[t] - ep_value_preds[t]
    advantages[t] = next_advantage = td_error + gamma * lam * ep_masks[t] * next_advantage

In [6]:
advantages.shape

NameError: name 'advantages' is not defined

In [7]:
returns.shape

NameError: name 'returns' is not defined

In [8]:
twohot_returns = torch.stack([twohot_encode(r) for r in returns])
critic_loss = - twohot_returns.detach() @ torch.log(batch_critic_dists).T
critic_loss = torch.sum(torch.diag(critic_loss))

NameError: name 'returns' is not defined

In [ ]:
# WORKS:
###def twohot_encode(x):
###    """
###    Computes the twohot encoded symlog returns.
###    
###    Args:
###        x (torch.Tensor): the original returns (scalar or vector of values) to be encoded
###            Shape: (B,)
###    
###    Returns:
###        encoded (torch.Tensor): the encoded symlog returns as a vector
###            Shape: (B, NUM_BUCKETS)
###    
###    Notes:
###        - note that the returns are automatically symlogged before the two-hot encoding
###    """
###    config = load_config()
###    
###    if not isinstance(x, torch.Tensor):
###        x = torch.tensor(x)
###    x = x.to(config["device"])
###    
###    # add an empty dim for scalar inputs
###    if not len(x.shape):
###        x = x.view(-1)
###        
###    buckets = torch.linspace(config["min_bucket"], config["max_bucket"], config["num_buckets"]).to(config["device"])
###    encoded = torch.zeros((x.shape[0], buckets.shape[0])).to(config["device"])
###    
###    for i in range(x.shape[0]):
###        # symlog transform the value
###        value = symlog(x[i])
###
###        if value < buckets[0]:
###            encoded[i, 0] = 1.0
###        elif value > buckets[-1]:
###            encoded[i, -1] = 1.0
###        else:
###            # index: right bucket (k+1 in the DreamerV3 paper), index-1: left bucket (k in the DreamerV3 paper)
###            index = torch.searchsorted(buckets, value, side="right")  # index: buckets[index-1] <= v < buckets[index]
###            weight = (buckets[index] - value) / (buckets[index] - buckets[index-1])
###            encoded[i, index-1] = weight
###            encoded[i, index] = 1 - weight
###
###    return encoded
###

In [24]:
twohot_encode(42)[159]

tensor(0.8448, device='cuda:0')

In [27]:
solution = torch.zeros(1,255).to(config["device"])
solution[0, 158] = 0.1552
solution[0, 159] = 0.8448

In [28]:
torch.isclose(twohot_encode(42), solution, atol=1e-4).all().item()

True

In [29]:
import torch

def twohot_encode(x):
    """
    Computes the twohot encoded symlog returns.
    
    Args:
        x (torch.Tensor): the original returns (scalar or vector of values) to be encoded
            Shape: (B,)
    
    Returns:
        encoded (torch.Tensor): the encoded symlog returns as a vector
            Shape: (B, NUM_BUCKETS)
    
    Notes:
        - note that the returns are automatically symlogged before the two-hot encoding
    """
    config = load_config()
    
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x)
    x = x.to(config["device"])
    
    # add an empty dim for scalar inputs
    if not len(x.shape):
        x = x.view(-1)
        
    buckets = torch.linspace(config["min_bucket"], config["max_bucket"], config["num_buckets"]).to(config["device"])
    
    # symlog transform the values
    values = symlog(x)

    # clip values at the borders to ensure they lie within the bucket range
    clipped_values = torch.clamp(values, buckets[0], buckets[-1])

    # calculate the index of the right bucket (k+1) and left bucket (k) for each value
    index = torch.searchsorted(buckets, clipped_values, side="right")
    left_bucket_index = torch.clamp(index - 1, 0, buckets.shape[0] - 1)
    right_bucket_index = torch.clamp(index, 0, buckets.shape[0] - 1)

    # calculate the weights
    weight = (buckets[right_bucket_index] - clipped_values) / (buckets[right_bucket_index] - buckets[left_bucket_index])
    weight = torch.clamp(weight, 0.0, 1.0)  # Ensure weights are within [0, 1]

    # create the one-hot encoding for each value
    encoded = torch.zeros((x.shape[0], buckets.shape[0])).to(config["device"])
    encoded.scatter_add_(1, left_bucket_index.unsqueeze(1), weight.unsqueeze(1))
    encoded.scatter_add_(1, right_bucket_index.unsqueeze(1), 1 - weight.unsqueeze(1))

    return encoded


In [30]:
buckets = torch.linspace(config["min_bucket"], config["max_bucket"], config["num_buckets"]).to(config["device"])

In [32]:
twohot_encode([-500.33, 42, 1, 2, 3]).shape

torch.Size([5, 255])

In [ ]:
twohot_encode(5).shape

In [53]:
symexp(twohot_encode(0.01) @ buckets)

tensor([0.0100], device='cuda:0')

In [ ]:
torch.tensor(5).view(-1).shape

In [36]:
result = symexp(twohot_encode([-500.33,10,0.42]) @ buckets)

In [37]:
solution = torch.tensor([-500.33,10,0.42]).to(config["device"])

In [38]:
result

tensor([-5.0033e+02,  1.0000e+01,  4.2000e-01], device='cuda:0')

In [39]:
solution

tensor([-5.0033e+02,  1.0000e+01,  4.2000e-01], device='cuda:0')

In [52]:
torch.isclose(result, solution).all().item()

True

In [ ]:
[twohot_encode(r) for r in returns]

In [ ]:
for key, val in episode_batches.items():
    print(key, val.shape)

In [ ]:
episode_batches["rewards"]